In [ ]:
# Things to do:
# how to scrape from a google docs with an API 
# Google docs plugin
# using BERTopic for visualisation: classify which are the most useful visualisations
# 

In [ ]:
# credits:
# https://understandingdata.com/posts/how-to-extract-the-text-from-multiple-webpages-in-python/

In [1]:
!pip install beautifulsoup4
!pip install numpy
!pip install requests
!pip install spacy
!pip install trafilatura

                                              0.0/1.0 MB ? eta -:--:--
     -                                        0.0/1.0 MB 640.0 kB/s eta 0:00:02
     ---                                      0.1/1.0 MB 871.5 kB/s eta 0:00:02
     --------                                 0.2/1.0 MB 1.7 MB/s eta 0:00:01
     ---------------                          0.4/1.0 MB 2.1 MB/s eta 0:00:01
     ---------------------                    0.5/1.0 MB 2.3 MB/s eta 0:00:01
     --------------------------               0.7/1.0 MB 2.4 MB/s eta 0:00:01
     -------------------------------          0.8/1.0 MB 2.6 MB/s eta 0:00:01
     ------------------------------------     1.0/1.0 MB 2.5 MB/s eta 0:00:01
     ---------------------------------------- 1.0/1.0 MB 2.6 MB/s eta 0:00:00
                                              0.0/41.9 kB ? eta -:--:--
     ---------------------------------------- 41.9/41.9 kB 2.0 MB/s eta 0:00:00
                                              0.0/41.0 kB ? eta -:--:--

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.27.1 requires charset-normalizer~=2.0.0; python_version >= "3", but you have charset-normalizer 3.2.0 which is incompatible.
stnet 1.0 requires numpy~=1.16.1, but you have numpy 1.21.5 which is incompatible.
stnet 1.0 requires pandas~=0.24.1, but you have pandas 1.5.3 which is incompatible.
stnet 1.0 requires scikit-image~=0.14.0, but you have scikit-image 0.19.3 which is incompatible.
stnet 1.0 requires scipy~=1.2.1, but you have scipy 1.9.3 which is incompatible.
stnet 1.0 requires torch~=1.0.0, but you have torch 1.12.1 which is incompatible.
stnet 1.0 requires torchvision~=0.2.1, but you have torchvision 0.13.1+cu113 which is incompatible.
stnet 1.0 requires tqdm~=4.26.0, but you have tqdm 4.45.0 which is incompatible.


In [4]:
from bs4 import BeautifulSoup
import json
import numpy as np
import requests
from requests.models import MissingSchema
import spacy
import trafilatura

In [5]:
urls = ["https://arxiv.org/abs/2304.06131","https://github.com/facebookresearch/segment-anything", "https://github.com/JerryX1110/awesome-segment-anything-extensions", 
"https://arxiv.org/abs/2304.06718", "https://github.com/UX-Decoder/Segment-Everything-Everywhere-All-At-Once" ]

In [6]:
data = {}

for url in urls:
    # 1. Obtain the response:
    resp = requests.get(url)
    
    # 2. If the response content is 200 - Status Ok, Save The HTML Content:
    if resp.status_code == 200:
        data[url] = resp.text

In [11]:
def beautifulsoup_extract_text_fallback(response_content):
    
    '''
    This is a fallback function, so that we can always return a value for text content.
    Even for when both Trafilatura and BeautifulSoup are unable to extract the text from a 
    single URL.
    '''
    
    # Create the beautifulsoup object:
    soup = BeautifulSoup(response_content, 'html.parser')
    
    # Finding the text:
    text = soup.find_all(text=True)
    
    # Remove unwanted tag elements:
    cleaned_text = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script',
        'style',]

    # Then we will loop over every item in the extract text and make sure that the beautifulsoup4 tag
    # is NOT in the blacklist
    for item in text:
        if item.parent.name not in blacklist:
            cleaned_text += '{} '.format(item)
            
    # Remove any tab separation and strip the text:
    cleaned_text = cleaned_text.replace('\t', '')
    return cleaned_text.strip()
    

def extract_text_from_single_web_page(url):
    
    downloaded_url = trafilatura.fetch_url(url)
    try:
        a = trafilatura.extract(downloaded_url, output_format ='json', with_metadata=True, include_comments = False,
                            date_extraction_params={'extensive_search': True, 'original_date': True})
    except AttributeError:
        a = trafilatura.extract(downloaded_url, output_format = 'json', with_metadata=True,
                            date_extraction_params={'extensive_search': True, 'original_date': True})
    if a:
        json_output = json.loads(a)
        return json_output['text']
    else:
        try:
            resp = requests.get(url)
            # We will only extract the text from successful requests:
            if resp.status_code == 200:
                return beautifulsoup_extract_text_fallback(resp.content)
            else:
                # This line will handle for any failures in both the Trafilature and BeautifulSoup4 functions:
                return np.nan
        # Handling for any URLs that don't have the correct protocol
        except MissingSchema:
            return np.nan

In [12]:
single_url = urls[0]
text = extract_text_from_single_web_page(url=single_url)
print(text)

Computer Science > Computer Vision and Pattern Recognition
[Submitted on 12 Apr 2023]
Title:UniverSeg: Universal Medical Image SegmentationDownload PDF
Abstract: While deep learning models have become the predominant method for medical image segmentation, they are typically not capable of generalizing to unseen segmentation tasks involving new anatomies, image modalities, or labels. Given a new segmentation task, researchers generally have to train or fine-tune models, which is time-consuming and poses a substantial barrier for clinical researchers, who often lack the resources and expertise to train neural networks. We present UniverSeg, a method for solving unseen medical segmentation tasks without additional training. Given a query image and example set of image-label pairs that define a new segmentation task, UniverSeg employs a new Cross-Block mechanism to produce accurate segmentation maps without the need for additional training. To achieve generalization to new tasks, we have g

In [13]:
urls = urls + ['fake_url']

text_content = [extract_text_from_single_web_page(url) for url in urls]

print(text_content[1])

Segment Anything
Alexander Kirillov, Eric Mintun, Nikhila Ravi, Hanzi Mao, Chloe Rolland, Laura Gustafson, Tete Xiao, Spencer Whitehead, Alex Berg, Wan-Yen Lo, Piotr Dollar, Ross Girshick
[
Paper] [
Project] [
Demo] [
Dataset] [
Blog] [
BibTeX]
The Segment Anything Model (SAM) produces high quality object masks from input prompts such as points or boxes, and it can be used to generate masks for all objects in an image. It has been trained on a dataset of 11 million images and 1.1 billion masks, and has strong zero-shot performance on a variety of segmentation tasks.
Installation
The code requires
python>=3.8, as well as
pytorch>=1.7 and
torchvision>=0.8. Please follow the instructions here to install both PyTorch and TorchVision dependencies. Installing both PyTorch and TorchVision with CUDA support is strongly recommended.
Install Segment Anything:
pip install git+https://github.com/facebookresearch/segment-anything.git
or clone the repository locally and install with
git clone git@gi

In [14]:
print(text_content[-1:])

[nan]


In [15]:
cleaned_textual_content = [text for text in text_content if str(text) != 'nan']


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.